In [1]:
import pandas as pd
path = '../MLtext2/MLtext2/data/yelp.csv'
yelp = pd.read_csv(path)

In [2]:
yelp.head(3)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0


In [3]:
yelp.loc[0,'text']

'My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.\n\nDo yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I\'ve ever had.  I\'m pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.\n\nWhile EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I\'ve ever had.\n\nAnyway, I can\'t wait to go back!'

In [4]:
yelp.stars.value_counts().sort_index()

1     749
2     927
3    1461
4    3526
5    3337
Name: stars, dtype: int64

In [5]:
yelp_best_worst = yelp[(yelp.stars==5) | (yelp.stars==1)]

In [6]:
yelp_best_worst.shape

(4086, 10)

In [7]:
X = yelp_best_worst.text
y = yelp_best_worst.stars

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
print(X_train.shape)
print(X_test.shape)

(3064,)
(1022,)


Part 2: Tokenizing the text

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [15]:
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
print(X_train_dtm.shape)
print(X_test_dtm.shape)

(3064, 16825)
(1022, 16825)


In [16]:
print(vect.get_feature_names()[-50:])

['yyyyy', 'z11', 'za', 'zabba', 'zach', 'zam', 'zanella', 'zankou', 'zappos', 'zatsiki', 'zen', 'zero', 'zest', 'zexperience', 'zha', 'zhou', 'zia', 'zihuatenejo', 'zilch', 'zin', 'zinburger', 'zinburgergeist', 'zinc', 'zinfandel', 'zing', 'zip', 'zipcar', 'zipper', 'zippers', 'zipps', 'ziti', 'zoe', 'zombi', 'zombies', 'zone', 'zones', 'zoning', 'zoo', 'zoyo', 'zucca', 'zucchini', 'zuchinni', 'zumba', 'zupa', 'zuzu', 'zwiebel', 'zzed', 'éclairs', 'école', 'ém']


In [17]:
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [18]:
vect=CountVectorizer(lowercase=False)
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

(3064, 20838)

In [20]:
vect = CountVectorizer(ngram_range=(1,2))
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm.shape

(3064, 169847)

In [21]:
print(vect.get_feature_names()[-50:])

['zone out', 'zone when', 'zones', 'zones dolls', 'zoning', 'zoning issues', 'zoo', 'zoo and', 'zoo is', 'zoo not', 'zoo the', 'zoo ve', 'zoyo', 'zoyo for', 'zucca', 'zucca appetizer', 'zucchini', 'zucchini and', 'zucchini bread', 'zucchini broccoli', 'zucchini carrots', 'zucchini fries', 'zucchini pieces', 'zucchini strips', 'zucchini veal', 'zucchini very', 'zucchini with', 'zuchinni', 'zuchinni again', 'zuchinni the', 'zumba', 'zumba class', 'zumba or', 'zumba yogalates', 'zupa', 'zupa flavors', 'zuzu', 'zuzu in', 'zuzu is', 'zuzu the', 'zwiebel', 'zwiebel kräuter', 'zzed', 'zzed in', 'éclairs', 'éclairs napoleons', 'école', 'école lenôtre', 'ém', 'ém all']


part 3: Comparing Accuracy

In [22]:
y_test.value_counts().head(1) / y_test.shape

5    0.819961
Name: stars, dtype: float64

In [26]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

def tokenize_test(vect):
    
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
    
    print('Features: ',X_train_dtm.shape[1])
    
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)
    y_pred_class = nb.predict(X_test_dtm)
    
    print('Accuracy: ',metrics.accuracy_score(y_test, y_pred_class))

In [27]:
vect = CountVectorizer()
tokenize_test(vect)

Features:  16825
Accuracy:  0.9187866927592955


In [28]:
vect = CountVectorizer(lowercase=False)
tokenize_test(vect)

Features:  20838
Accuracy:  0.9099804305283757


In [29]:
vect = CountVectorizer(ngram_range=(1,2))
tokenize_test(vect)

Features:  169847
Accuracy:  0.8542074363992173


Part 4: Removing frequent terms

In [30]:
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [32]:
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  16528
Accuracy:  0.9158512720156555


In [33]:
print(sorted(vect.get_stop_words()))

['a', 'about', 'above', 'across', 'after', 'afterwards', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'amoungst', 'amount', 'an', 'and', 'another', 'any', 'anyhow', 'anyone', 'anything', 'anyway', 'anywhere', 'are', 'around', 'as', 'at', 'back', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'below', 'beside', 'besides', 'between', 'beyond', 'bill', 'both', 'bottom', 'but', 'by', 'call', 'can', 'cannot', 'cant', 'co', 'con', 'could', 'couldnt', 'cry', 'de', 'describe', 'detail', 'do', 'done', 'down', 'due', 'during', 'each', 'eg', 'eight', 'either', 'eleven', 'else', 'elsewhere', 'empty', 'enough', 'etc', 'even', 'ever', 'every', 'everyone', 'everything', 'everywhere', 'except', 'few', 'fifteen', 'fifty', 'fill', 'find', 'fire', 'first', 'five', 'for', 'former', 'formerly', 'forty', 'found', 'four', 'from', 'front', 'full', 'further', 'get', 'give

In [34]:
vect = CountVectorizer(max_df=0.5)
tokenize_test(vect)

Features:  16815
Accuracy:  0.9207436399217221


In [36]:
#print words that were removed by max_df
print(vect.stop_words_)

{'it', 'in', 'for', 'my', 'this', 'to', 'and', 'is', 'the', 'of'}


Part 5: Removing infrequent terms

In [37]:
vect = CountVectorizer(max_features=1000)
tokenize_test(vect)

Features:  1000
Accuracy:  0.8923679060665362


In [38]:
vect = CountVectorizer(min_df=2)
tokenize_test(vect)

Features:  8783
Accuracy:  0.9246575342465754


In [39]:
vect = CountVectorizer(ngram_range=(1,2), min_df=2)
tokenize_test(vect)

Features:  43957
Accuracy:  0.9324853228962818


Part 6: Handling Unicode Errors

Text is made of characters, but files are made of bytes. These bytes represent some characters according to some encoding. 

In [40]:
print(type('hello'))
print(type(u'hello'))

<class 'str'>
<class 'str'>


In [42]:
print(type(b'hello'))
print(type('hello'))

<class 'bytes'>
<class 'str'>


In [48]:
b'hello'.decode(encoding='utf-8')

'hello'

In [49]:
'hello'.encode(encoding='utf-8')

b'hello'

In [60]:
path = '../MLtext2/MLtext2/data/yelp_single.csv'
single = pd.read_csv(path, encoding='latin-1')
single.head()

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,-vaGR2CXNS4NZrMdctSkGA,2011-08-07,8VN65nUBl22-g81bcSeLDA,4,The first thing to know about this Los Reyes l...,review,ikm0UCahtK34LbLCEw4YTw,3,4,2
